## read pose info and find ball start and end points 

## with single input in function 

In [6]:
import json
import math
import cv2
import os 
file = "/home/jakhon37/PROJECTS/super_shot/examples/res/alphapose-results.json" #./data/txt/alphapose-results.json"
path = '/home/jakhon37/PROJECTS/super_shot/examples/res/vis/'
file = "./data/res/alphapose-results.json" #./data/txt/alphapose-results.json"
path = './data/res/vis/'

ball_center = [[710,1580]]
ball_center = [[536, 1054]]

ball_c_y = ball_center[0][1]
ball_c_x = ball_center[0][0]
ball_center = ball_center[0]
end_pointls = []
start_pointls = []
listt = []
tr_angles = []
# for root, dr, files in os.walk(path):
#     print('root ', root, 'dir ', dr, 'files', files)

def normalize_nnum(list_in, q_range):
    list_normilized = []
    for idx, i in enumerate(list_in):
        Q, xi, x = q_range, list_in[idx], list_in
        zi = (xi - min(x)) / (max(x) - min(x)) * Q
        list_normilized.append(zi)
    return list_normilized

def normalize_single_nnum(list_in, q_range, in_nnum):
    list_normilized = []
    list_in.append(in_nnum)
    for idx, i in enumerate(list_in):
        Q, xi, x = q_range, list_in[idx], list_in
        zi = (xi - min(x)) / (max(x) - min(x)) * Q
        list_normilized.append(zi)
    return list_normilized[-1]
    
def pose_calculation(json_f, path, ball_center):
    
    
    json_f = json_f
    im3_path = path + json_f["image_id"]
    im3 = cv2.imread(im3_path)
    im3_shape = im3.shape
    print('image shape ',im3_shape)

    ball_center = json_f["ball_center"]
    print(ball_center)
    # pose infos 
    print(json_f["image_id"])
    print('.....................')

    # person center: calculate middle x point between right and left ankles
    av_ank_point = abs(json_f["keypoints"][48] - json_f["keypoints"][45]) / 2 + min(json_f["keypoints"][45],json_f["keypoints"][48])

    # calculate distance between right knee and ankle for ratio
    difx = abs(json_f["keypoints"][42] - json_f["keypoints"][48]) # kneeX - ankleX 
    dify = abs(json_f["keypoints"][43] - json_f["keypoints"][49]) # kneeY - ankleY 
    dif_c = math.sqrt(difx**2+dify**2)
    org = 100
    ratio = org/dif_c

    # real size by ratio 
    toe_difx = (json_f["keypoints"][60] - json_f["keypoints"][63])
    real_toe_difx = toe_difx * ratio

    # find angle to rotate centeral line
    angle_l = 25*(35-real_toe_difx)/51
    # tr_angles.append(angle_l)
    print('angle found: ',angle_l, 'with dif value ', real_toe_difx)
    # find line start and end points
    im_h , im_w = im3_shape[0] , im3_shape[1] # image hight width
    scale1 = 0.5#0.384 # used to set last y point for line 
    scale = dif_c

    y_point = json_f["keypoints"][52] # person head point 
    b_line = ball_center[1] - y_point #  im_h*scale
    # print('b line ', b_line)
    # print('b y_point ', y_point)
    if angle_l>0:
        a_line = math.tan(math.radians(angle_l))*b_line
        end_point = int(ball_center[0] -a_line), int(y_point) # x, y end points 
    else: 
        a_line = math.tan(math.radians(-angle_l))*b_line
        end_point = int(ball_center[0] +a_line), int(y_point) # x, y end points 
#         end_pointls.append(end_point)
    start_p_y = im_h*0.9 # used to set start y point by scaling 
    # start_p_y = (im_h - ball_center[0][1])*scale+ball_center[0][1]
    # find x start point 
    bs_line = start_p_y - ball_center[1]
    if angle_l>0:
        as_line = math.tan(math.radians(angle_l))*bs_line
        start_point = int(ball_center[0] + as_line), int(start_p_y) # x, y start points
    else:
        as_line = math.tan(math.radians(-angle_l))*bs_line
        start_point = int(ball_center[0] - as_line), int(start_p_y) # x, y start points
    start_point = ball_center# = [536, 1054]
    
    list_guide_toe_dif = [112.73243410035097, 100.79666125996108, 147.74704350886483, 173.44834966030902, 213.39849046056747, 167.9547926888052]
    out_dif = normalize_single_nnum(list_guide_toe_dif, 10, dif_c)
    
    list_guide_angles = [15.202324038111689, -8.763352914537702, -0.676629864061838, 7.899643145846709, 24.944131204114143, 11.31640899790251]
    out_angle = normalize_single_nnum(list_guide_angles, 5, angle_l)
    
    zi = out_angle
    end_pointl =[end_point[0], int(end_point[1]+end_point[1]*zi/100)] # trajectory end point 
    height_y_trj = (start_point[1]- end_pointl[1]) # use for setting trajectory height on y axise 
    print(height_y_trj,(start_point[1],end_pointl[1]),ratio)

    list_guide_haight = [341, 345, 486, 529, 648, 491]
    out_height = normalize_single_nnum(list_guide_haight, 10, height_y_trj)
    
    if not os.path.exists(path):
        os.makedirs(path)
    im3_path = path + json_f["image_id"]
    im3 = cv2.imread(im3_path)
    cv2.line(im3, start_point, end_pointl, color=(255, 50, 50 ), thickness=3)
    cv2.imwrite((f'{path}out{json_f["image_id"]}'), im3)
 
    print('')
    print('out path - ', path)
    return out_angle, out_dif, end_pointl, out_height, ratio, angle_l 

if __name__ == "__main__":
    
    with open(file, "r") as f: 
        json_f = json.load(f)
        file = json_f[1]
        out_angle, out_dif, end_pointl, out_height, ratio, angle_l  = pose_calculation(file, path, ball_center)

            
#         hhh = []
#         for i in json_f:
#             file = i
#             out_angle, out_dif, end_pointl, height_y_trj, ratio, angle_l = pose_calculation(file, path, ball_center)
#             list_of_outputs = [out_angle, out_dif, end_pointl, height_y_trj]
#             hhh.append(height_y_trj)
#     print(hhh)
    
    print('output: ', out_angle, out_dif, end_pointl, out_height, ratio, angle_l )

    

image shape  (1920, 1080, 3)
[535, 1055]
im3.jpg
.....................
angle found:  -8.763352914537702 with dif value  52.87723994565691
345 (1055, 710) 0.9920963526965795

out path -  ./data/res/vis/
output:  0.0 0.0 [588, 710] 0.13029315960912052 0.9920963526965795 -8.763352914537702


In [122]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def main_dist_time(vel, angle, angle_l):
    spin = abs(angle-8)*250 + 2000
    # Drag coefficient, projectile radius (m), area (m2) and mass (kg).
    c = 0.47           # drag coefficent -> An average golf ball will have a drag coefficient of, 0.24 < CD < 0.7
    r = 0.02135         # radius diameter(m) = 42.7mm --> r=d/2 convert to meter = 0.02135
    A = np.pi * r**2    # area
    m = 0.04593         # mass(kg), golf_ball = 45.93 grams
    rho_air = 1.28    # Air density (kg.m-3), acceleration due to gravity (m.s-2).
    g = 9.81
    k = 0.5 * c * rho_air * A
    v0,vel=vel*2,vel*2
    phi0 = np.radians(angle)
    def deriv(t, u):
        x, xdot, z, zdot = u
        speed = np.hypot(xdot, zdot)
        xdotdot = -k/m * speed * xdot
        zdotdot = -k/m * speed * zdot - g
        return xdot, xdotdot, zdot, zdotdot
    # Initial conditions: x0, v0_x, z0, v0_z.
    u0 = 0, v0 * np.cos(phi0), 0., v0 * np.sin(phi0)
    # Integrate up to tf unless we hit the target sooner.
    t0, tf = 0, 50
    def hit_target(t, u):
        # We've hit the target if the z-coordinate is 0.
        return u[2]
    # Stop the integration when we hit the target.
    hit_target.terminal = True
    # We must be moving downwards (don't stop before we begin moving upwards!)
    hit_target.direction = -1
    def max_height(t, u):
        # The maximum height is obtained when the z-velocity is zero.
        return u[3]
    soln = solve_ivp(deriv, (t0, tf), u0, dense_output=True, events=(hit_target, max_height))
    # A fine grid of time points from 0 until impact time.
    t = np.linspace(0, soln.t_events[0][0], 1000)
    # Retrieve the solution for the time grid and plot the trajectory.
    sol = soln.sol(t)
    x, z = sol[0], sol[2]
    top_h = max(z)
    # creating .obj (3D) file...
    f = open("trajectory.obj", "w")
    local_path = './data/txt/'     
#    num_file = "pointsNumpy_tr.npy"
    np_file = f'{local_path}pointsNumpy_tr'
    np_list = []



    for i in range(x.shape[0]):
        
        xxx = x[i]
        if xxx ==0:
            xxx = 1
        if angle_l < 0:
            zz = z[i]/xxx
        else:
            zz = -z[i]/3
        points = f"v {x[i]} {zz} {z[i]}\n"
        f.write(points)
        points = [x[i], z[i],  zz]
        arrPoints = np.array(points) 
        np_list.append(arrPoints)
    
    np.save(np_file, np_list)
    f.close()
#     x[-1], top_h = check_angle_spin(angle, vel, x[-1], top_h)
   # print(x[-1], top_h)
    return x[-1], top_h, t[-1]


In [130]:
if __name__ == "__main__":

    vel, angle = 90, 21
    distance, height, time = main_dist_time(vel, angle, angle_l)
    print(distance, height, time)

238.1123058341294 44.06773337432843 5.715616713440645


## read the numpy file and rotate the object and save json file

In [124]:
import numpy as np 
import math as m 
import json
 
def Rx(theta): 
    return np.matrix([[ 1, 0           , 0           ], 
                   [ 0, m.cos(theta),-m.sin(theta)], 
                   [ 0, m.sin(theta), m.cos(theta)]]) 
 
def Ry(theta): 
    return np.matrix([[ m.cos(theta), 0, m.sin(theta)], 
                   [ 0           , 1, 0           ], 
                   [-m.sin(theta), 0, m.cos(theta)]]) 
 
def Rz(theta): 
    return np.matrix([[ m.cos(theta), -m.sin(theta), 0 ], 
                   [ m.sin(theta), m.cos(theta) , 0 ], 
                   [ 0           , 0            , 1 ]]) 
 
# ------------------------------------------------------------------------------ 


def rotate_ball(tr_x, angle, points, local_path):
#     for tr_x, tr_z, points in zip(tr_x_list, tr_angles, points_all):
    tr_z = angle

    degreey = 90+0#tr_z
    degreex = 4+tr_x#15+tr_x  # minimum degree for x is set to 13
    if tr_z> 15.5:
        tr_z = tr_z - tr_z*0.50

    degreez = -9# -tr_z # tr_z #tr_angles[4]


#     print(points)

    f = open(f'{local_path}3dtr_{degreey}.obj', 'w') 
    r_json = {}
    r_json["parabola"] = [ ] 
    
    points_x = []
    points_z = []
    scale = 20
    for i in range(points.shape[0]): 
#         print(points)
        vec = points[i] * scale
        rotated_vec = Ry(m.radians(degreey)) * vec.reshape(3, 1) 
        # print(rotated_vec.shape) 
    #     r_json["parabola"].append([rotated_vec[0,0], rotated_vec[1,0], rotated_vec[2,0]]) 


    #     f.write(f'v {rotated_vec[0,0]} {rotated_vec[1,0]} {rotated_vec[2,0]} {200} {200} {200}\n') 
        points_x.append([rotated_vec[0,0], rotated_vec[1,0], rotated_vec[2,0]]) 

    points_x = np.array(points_x) 

    for i in range(points_x.shape[0]): 
    #     print(i) 
        vec = points_x[i] 
        rotated_vec = Rx(m.radians(degreex)) * vec.reshape(3, 1) 
        # print(rotated_vec.shape) 
        points_z.append([rotated_vec[0,0], rotated_vec[1,0], rotated_vec[2,0]]) 

    points_z = np.array(points_z) 


    for i in range(points_z.shape[0]): 
    #     print(i) 
        vec = points_z[i] 
        rotated_vec = Rz(m.radians(degreez)) * vec.reshape(3, 1) 


        r_json["parabola"].append([rotated_vec[0,0], rotated_vec[1,0], rotated_vec[2,0]]) 


        f.write(f'v {rotated_vec[0,0]} {rotated_vec[1,0]} {rotated_vec[2,0]} {200} {200} {200}\n')

    f.close()
    multikeys.append(r_json)
    json_f = f"{local_path}json_file1.json"
    json_f2 = f"{local_path}json_file2.json"
    s = json.dumps(multikeys,indent="\t")
    open(json_f2,"w").write(s)

    with open(json_f, "a") as f: 
        json.dump(r_json, f, indent="\t") 


    print('done')
    
if __name__ == "__main__":
#     for tr_x, tr_z, points in zip(tr_x_list, tr_angles, points_all):

    multikeys=[]
    local_path = './data/txt/'     
    points = np.load((local_path+'pointsNumpy_tr.npy'))#, allow_pickle=True) 

    rotate_ball(out_height, angle_l, points, local_path)


done


In [125]:
path = 'data/vid/vid1.mp4'
vid = "bvid.mp4"
p = f'{"/".join(path.split("/")[0:-1])}/{vid}'
c = path.split("/")[-1].split('.')[0]
print(c)
print(p)

vid1
data/vid/bvid.mp4


## read video and json file and draw the trajectory values on video 

In [126]:
def draw_polylines(frame, coords_list, RGB):
    coords_list = np.array(coords_list)
    frame = cv2.polylines(
        frame, [coords_list],
        isClosed=False, color=(int(RGB[2]), int(RGB[1]), int(RGB[0])),
        thickness=10)
    coords_list = np.ndarray.tolist(coords_list)
    return coords_list, frame

def draw_guide_lines(frame, coords_list, RGB):
    coords_list = np.array(coords_list)
    frame = cv2.polylines(
        frame, [coords_list],
        isClosed=False, color=(int(RGB[2]), int(RGB[1]), int(RGB[0])),
        thickness=10)
    coords_list = np.ndarray.tolist(coords_list)
    return coords_list, frame


def draw_circle(frame, center_coordinate):
    frame = cv2.circle(frame, center_coordinate, radius=5, color=(0,255,250), thickness=15)
    
def points_between(x1, y1, x2, y2):
    d0 = x2 - x1
    d1 = y2 - y1
    count = max(abs(d1+1), abs(d0+1))
    if d0 == 0:
        return (np.full(count, x1), np.round(np.linspace(y1, y2, count)).astype(np.int32))
    if d1 == 0:
        return (np.round(np.linspace(x1, x2, count)).astype(np.int32), np.full(count, y1), )
    return (np.round(np.linspace(x1, x2, count)).astype(np.int32), np.round(np.linspace(y1, y2, count)).astype(np.int32))


import os
import numpy as np
import cv2
import json
path = 'data/'
path_v = path + 'vid/vid1.mp4'
jsonfile = path + "txt/json_file1.json"


def draw_main(json_f, path_v, vid_id, st_points, intl_pos):
    vid_path = (f'{path_v}{vid_id.split(".")[0]}.mp4')
    cap = cv2.VideoCapture(vid_path)  # read video 
    # cap = cv2.VideoCapture(0)
    if (cap.isOpened()== False):   # Check if camera opened successfully
        print("Error opening video stream or file")

    RGB = np.random.randint(0, 256, size=3)  # 0~255 의 숫자 3개를 랜덤하게 가져옴
    RGB = (0, 256, 3)
    video_fps = cap.get(cv2.CAP_PROP_FPS),
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    print("video: ", vid_id)
    print(f" Frame Per second: {video_fps } \n Total Frames: {total_frames} \n Height: {height} \n Width: {width}")

    # Define the codec and create VideoWriter objeact
    width, height = 1080, 1920   # assign nnew video width and height 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # *'mp4v' *'XVID'
    vid_out = f'{path_v}{vid_id.split(".")[0]}out.mp4'
    out = cv2.VideoWriter(vid_out,fourcc, 20.0, (width,height)) #(int(cap.get(3)),int(cap.get(4)))) # size (int(cap.get(3)),int(cap.get(4)))

#     jsonfile = path + "txt/json_file1.json"
    # jsonfile = "json_file.json"
#     with open(jsonfile, "r") as f: 
#         json_f = json.load(f)
    pr = json_f #["parabola"] # list of coordinates in format [[x,y,z],[x,y,z],...]
  
                
                
    scale = 1 # scale coordinates to normalize line projectile on video / find it from pose estimation 
    cur2 = 0
    cur = 1
    cur_n = 1 # 10
    cur_m = 1 # 130
    coords_list_X = []
    coords_lt_X = []
    coords_list =[]
    center_coordinates = st_points # 536, 1053 # 450, 1300
    initaial_pos = intl_pos - 2 # 60 # 62 / intl - 2
    
    coords_l_X = ([int(pr[0][0]*scale+center_coordinates[0]), int(pr[0][1]*(-scale)+center_coordinates[1])],
                        [int(pr[-1][0]*scale+center_coordinates[0]), int(pr[-1][1]*(-scale)+center_coordinates[1])] )
                
    z_line_points, z_line_points2 = points_between(x1=coords_l_X[0][0], y1=coords_l_X[0][1], x2=coords_l_X[1][0], y2=coords_l_X[1][1])
    for xx, zz in zip(z_line_points,z_line_points2):
        # print(xx)
        coords_list_X.append([xx,zz])
    # print(coords_list_X)
    # print(coords_list_X[0])
    ee =1
    while(cap.isOpened()):
        cur_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        ret, frame = cap.read()            
        frame = cv2.flip(frame,-1)
        if frame is None:
            break
        # imgResult = img.copy()
        overlay = frame.copy()
        overlay2 = frame.copy()


        if ret==True:
            frame = frame[0:height, 0:width]   # resize video
            if cur_frame <initaial_pos:
                cord_x, cord_y, cord_z = int(pr[0][0]*scale+center_coordinates[0]), int(pr[0][1]*(-scale)+center_coordinates[1]), int(pr[0][2]*(-scale))#+center_coordinates[2])
                coords_list.append([cord_x, cord_y] )
                center_coord = cord_x, cord_y
            if cur_frame > initaial_pos and len(pr)>cur:
                cur = ee
                cord_x, cord_y, cord_z = int(pr[cur][0]*scale+center_coordinates[0]), int(pr[cur][1]*(-scale)+center_coordinates[1]), int(pr[cur][2]*(-scale))#+center_coordinates[2])
                coords_list.append([cord_x, cord_y] )
                ee+=29

               
                
                center_coord = cord_x, cord_y
                # cur_m = 35
                if cur_n>cur_m:
                    cur_n = 50
                cur = abs(int(cur + cur_m+cur_n))
                
    #             print(cur)

                
                ###################
                total_frames
                total_3dcords = len(pr)
                frm_ratio = cur/total_3dcords
                zratio = int(len(coords_list_X)*frm_ratio)
                if zratio>=len(coords_list_X):
                    zratio = len(coords_list_X)-1
                zz_xx = coords_list_X[zratio]
                coords_lt_X.append(zz_xx)
                
                
                    
                    
                ##########################################
            coords_list, frame = draw_polylines(frame, coords_list, RGB)
            coords_lt_X, overlay = draw_guide_lines(overlay, coords_lt_X, (255, 100, 100))
            
            # print(cur_frame)
            # print(total_frames)
            # print(initaial_pos)
            # rat_al = cur_frame/(total_frames) #+(initaial_pos-2)
            alpha = 0.45 #= 1-rat_al
            
            # Perform weighted addition of the input image and the overlay
            # frame = cv2.addWeighted(overlay2, alpha, frame, 1 - alpha, 0)
            frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
            image = draw_circle(frame, center_coord)
#             cur_m = cur_m/1.045
#             cur_n = cur_n*1.0002
            cur_m = cur_m/1.45
            cur_n = cur_n*1.002
            out.write(frame)
            sc = 0.4 # rezie ratio for video out display 
            frame = cv2.resize(frame,(int(width*sc), int(height*sc)),fx=0,fy=0, interpolation = cv2.INTER_CUBIC) 
            cv2.imshow('frame',frame)
            # img_path = f'{"/".join(path_v.split("/")[0:-1])}/img{path_v.split("/")[-1].split(".")[0]}'
            img_path = f'{path_v}/IM{vid_id.split(".")[0]}'
            if not os.path.exists(img_path):
                os.makedirs(img_path)
            cv2.imwrite((img_path + "/image_{}.jpg".format(cur_frame)), frame)  # Save image localy 

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [127]:
import os

# List all files in a directory using os.listdir
path_vid = './data/vid/'
basepath_txt = 'data/txt/'

file = "./data/res/alphapose-results.json" #./data/txt/alphapose-results.json"
with open(file, "r") as f: 
    json_pose = json.load(f)
    
file_js = os.path.join(basepath_txt, 'json_file2.json')
with open(file_js, "r") as f: 
    json_f = json.load(f)

    
i = 1
stt_points = json_pose[i]["ball_center"]
im_id = json_pose[i]["image_id"]
json_trj = json_f[0]["parabola"]
print(json_trj[i][2])
intl_pos = json_pose[i]["initial_position"]
    
draw_main(json_trj,path_vid,im_id, stt_points, intl_pos)

        
        
# for i in range(len(json_pose)):
#     stt_points = json_pose[i]["ball_center"]
#     im_id = json_pose[i]["image_id"]
#     json_trj = json_f[i]["parabola"]
#     print(json_trj[i][2])
#     intl_pos = json_pose[i]["initial_position"]
#     draw_main(json_trj,path_vid,im_id, stt_points, intl_pos)

-13.131290199175544
video:  im3.jpg
 Frame Per second: (30.007370231284877,) 
 Total Frames: 209.0 
 Height: 1920.0 
 Width: 1080.0


In [63]:
import os

# List all files in a directory using os.listdir
path_vid = './data/vid/'
basepath_txt = 'data/txt/'

file = "./data/res/alphapose-results.json" #./data/txt/alphapose-results.json"
with open(file, "r") as f: 
    json_pose = json.load(f)
    
file_js = os.path.join(basepath_txt, 'json_file2.json')
with open(file_js, "r") as f: 
    json_f = json.load(f)

for i in range(len(json_pose)):
    stt_points = json_pose[i]["ball_center"]
    im_id = json_pose[i]["image_id"]
    json_trj = json_f[i]["parabola"]
    print(json_trj[i][2])
    intl_pos = json_pose[i]["initial_position"]
    draw_main(json_trj,path_vid,im_id, stt_points, intl_pos)


0.0
video:  im2.jpg
 Frame Per second: (30.007554644558898,) 
 Total Frames: 237.0 
 Height: 1920.0 
 Width: 1080.0


IndexError: list index out of range

In [57]:
def points_between(x1, y1, x2, y2):
    d0 = x2 - x1
    d1 = y2 - y1
    count = max(abs(d1+1), abs(d0+1))
    if d0 == 0:
        return (np.full(count, x1), np.round(np.linspace(y1, y2, count)).astype(np.int32))
    if d1 == 0:
        return (np.round(np.linspace(x1, x2, count)).astype(np.int32), np.full(count, y1), )
    return (np.round(np.linspace(x1, x2, count)).astype(np.int32), np.round(np.linspace(y1, y2, count)).astype(np.int32))

In [58]:
points_between(x1=0, y1=100, x2=50, y2=50)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]),
 array([100,  99,  98,  97,  96,  95,  94,  93,  92,  91,  90,  89,  88,
         87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,  75,
         74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,  62,
         61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50]))

In [2]:
def draw_line(im, start, end):
    x1,y1,x2,y2 = start[0], start[1], end[0], end[1]
    im1 = ImageDraw.Draw(im)
    im1.line((x1,y1,x2,y2), fill=(255, 255, 255, 128), width=1 )
    return im #out

def draw_rect(im, start, end, op):
    x1,y1,x2,y2 = start[0], start[1], end[0], end[1]
    im2 = Image.new("RGBA", im.size, (255, 255, 255, 0))
    im1 = ImageDraw.Draw(im2)
    im1.rectangle(((x1,y1),(x2,y2)), outline = (55, 55, 55, 0), fill=(0, 0, 20, op), width=10 )
    im2 = Image.alpha_composite(im, im2)
    return im2

def draw_arc(im, start_an, end_an, shape):
    # shape = [(40, 40), (145, 145)] start = 45, end = 135
    x1,y1,x2,y2 = start_an[0], start_an[1], end_an[0], end_an[1]
    im_copy = Image.new("RGBA", im.size, (255, 255, 255, 0))
    im = ImageDraw.Draw(im_copy)
    im.arc(shape, start_an, end_an, fill ="pink", width=200)
    out = Image.alpha_composite(im, im_copy)
    return out

def draw_txt(im, fnt_path, point, word, color, size, ops):
    # fnt_path = "/Users/jakhon37/pyProjects/3dRotation/data/res/font/Amble-Bold.ttf"
    fnt = ImageFont.truetype(fnt_path, size)
    x,y = point[0], point[1]
    im_copy = Image.new("RGBA", im.size, (255, 255, 255, 0))
    im1 = ImageDraw.Draw(im_copy)
    im1.text((x, y), word, font=fnt, fill=(color[0],color[1], color[1], ops))
    out = Image.alpha_composite(im, im_copy)
    return out

def put_img(im1, im2):
    im1.paste(im2, (37, 30), im2)
    return im1

In [9]:
from PIL import Image, ImageDraw, ImageFont

def main_draw(im, im2, distance, velocity, angle, carry_d, t_time, spin_rpm, height_tr, type_tr):
#    im = Image.fromarray(im).convert("RGBA")
#    im2 = Image.fromarray(im2).convert("RGBA")
    height_tr = int(height_tr)
    velocity   = int(velocity)
    t_time   = round(t_time, 2)
    angle   = int(angle)
    carry_d   = int(carry_d)
    spin_rpm = str(int(spin_rpm))
    fnt = ("data/res/font/Amble-Bold.ttf")
    w, h = im.size                    #  image_5.jpg

    # distance, velocity, angel = ["math_out"]["solutions"]["solution2_1"]["ddistance"], json_data["math_out"]["solutions"]["solution2_1"]["velocity"], json_data["math_out"]["solutions"]["solution2_1"]["angle"]
    
    ####  rectangle  ####
    start, end = (5, h-860), (305, 5)
    im = draw_rect(im, start, end, op=180)

    ####  put img  ####
    im = put_img(im, im2)

    ####  line  ####
    start, end = (35, h-1800), (275, h-1800) # first top line 
    im = draw_line(im, start, end)
    #### below the yard
    start, end = (35, h-1600), (275, h-1600)
    im = draw_line(im, start, end)
    ### between carry / avg total
    start, end = (155, h-1475), (155, h-1595)
    im = draw_line(im, start, end)
    ### below carry / time
    start, end = (35, h-1470), (275, h-1470)
    im = draw_line(im, start, end)

    #### below the M/S
    start, end = (35, h-1270), (275, h-1270)
    im = draw_line(im, start, end)
    ### between ANGEL / avg BACKSPIN
    start, end = (155, h-1145), (155, h-1265)
    im = draw_line(im, start, end)
    ### anle / spin 
    start, end = (35, h-1140), (275, h-1140)
    im = draw_line(im, start, end)
    ### between BACK/UP SPIN / SIDE SPIN 
    start, end = (155, h-1005), (155, h-1135)
    im = draw_line(im, start, end)
    ### below the TIME
    start, end = (35, h-995), (275, h-995)
    im = draw_line(im, start, end)
    
    ####  put txt  ####
    #### distance
    distance = int(distance)
    if len(str(distance)) > 2:
        point = (45, h-1780)
    elif len(str(distance )) == 2:
        point = (80, h-1780)
    elif len(str(distance )) <2:
        point = (120, h-1780)
    point2, word, color, size, ops = (45, h-1780), f'{distance}', (255, 255, 255), 120, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    #### total distance 
    distance = 'DISTANCE'
    point, word, color, size, ops = (110, h-1790), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    #### yards
    distance = 'YARDS'
    point, word, color, size, ops = (100, h-1650), f'{distance}', (255, 255, 255), 35, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    
    #### velocity 
    distance = 'VELOCITY'
    point, word, color, size, ops = (47, h-1580), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
        ####  VELOCITY value
    distance = carry_d
    if len(str(distance)) > 2:
        point = (45, h-1550)
    elif len(str(distance )) == 2:
        point = (80, h-1550)
    elif len(str(distance )) < 2:
        point = (75, h-1550)
    point2, word, color, size, ops = (35, h-1550), f'{distance}', (255, 255, 255), 60, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    
    ####  height 
    distance = 'APEX'
    point, word, color, size, ops = (190, h-1580), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    ####   apex height 
    distance =  height_tr
    if len(str(distance)) > 1:
        point = (183, h-1550)
    elif len(str(distance )) <=1:
        point = (197, h-1550)
    point2, word, color, size, ops = (193, h-1550), f'{distance}', (255, 255, 255), 60, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)

    ####  TOTAL TIME 
    distance = 'TOTAL TIME'
    point, word, color, size, ops = (95, h-1460), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    ####  TOTAL TIME 
    distance = velocity
    if len(str(velocity)) > 2:
        point = (65, h-1450)
    elif len(str(velocity)) == 2:
        point = (80, h-1450)
    elif len(str(velocity)) < 2:
        point = (115, h-1450)
    word, color, size, ops = f'{distance}', (255, 255, 255), 120, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    #### total dime indicator
    distance = 'seconds'
    point, word, color, size, ops = (90, h-1320), f'{distance}', (255, 255, 255), 35, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    
    #### laUnch angle 
    distance = 'LAUNCH ANGLE'
    point, word, color, size, ops = (57, h-1250), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    #### launch angle value
    distance = angle
    if len(str(distance)) > 1:
        point = (53, h-1220)
    elif len(str(distance)) <= 1:
        point = (68, h-1220)
    point2, word, color, size, ops = (53, h-1220), f'{distance}', (255, 255, 255), 60, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    
    #### DIRECTION angle 
    distance = 'DIRECTION'
    point, word, color, size, ops = (173, h-1250), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    ####  direction angle value
    distance = angle
    if len(str(distance )) <=1:
        point, word = (197, h-1220),  f'{distance}'
    direct_l, direct_r, direct_st = 'L', 'R', 'SR'
    if len(str(distance)) > 1:
        if distance >0.5:
            point, word = (173, h-1220),  f'{direct_r}'
            point, word = (183, h-1220),  f'{distance}'
        elif distance <-0.5:
            point, word = (173, h-1220),  f'{direct_l}'
            point, word = (183, h-1220),  f'{distance}'
        elif distance <0.5 and distance >-0.5:
            point, word = (173, h-1220),  f'{direct_st}'
            point, word = (183, h-1220),  f'{distance}'
    point2, word2, color, size, ops = (53, h-1220), f'{distance}', (255, 255, 255), 60, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    
       #### angle 
    distance = 'BACK SPIN'
    point, word, color, size, ops = (57, h-1130), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
        ####  back spin value
    distance = spin_rpm[0]
    if len(str(spin_rpm[1:])) > 1:
        point = (53, h-1100)
    elif len(str(spin_rpm[1:])) <= 1:
        point = (68, h-1100)
    point2, word, color, size, ops = (170, h-1220), f'{distance}', (255, 255, 255), 60, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    distance = (f',{spin_rpm[1:]}')
    if len(str(spin_rpm[1:])) > 1:
        distance = (f',{spin_rpm[1:]}')
    elif len(str(spin_rpm[1:])) <= 1:
        distance = (f' ')
    #### side spin 
    distance = 'SIDE SPIN'
    point, word, color, size, ops = (173, h-1130), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    ####  side spin value
    distance = spin_rpm[0]
    if len(str(spin_rpm[1:])) > 1:
        point = (170, h-1100)
    elif len(str(spin_rpm[1:])) <= 1:
        point = (200, h-1100)
    point2, word, color, size, ops = (170, h-1220), f'{distance}', (255, 255, 255), 60, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    distance = (f',{spin_rpm[1:]}')
    if len(str(spin_rpm[1:])) > 1:
        distance = (f',{spin_rpm[1:]}')
    elif len(str(spin_rpm[1:])) <= 1:
        distance = (f' ')
    point, word, color, size, ops = (200, h-1205), f'{distance}', (255, 255, 255), 45, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
#     ####  time 
#     distance = t_time
#     point, word, color, size, ops = (35, h-1120), f'{distance}', (255, 255, 255), 120, 255
#     im = draw_txt(im, fnt, point, word, color, size, ops)
#     ####  total time 
#     distance = 'TOTAL TIME'
#     point, word, color, size, ops = (100, h-1130), f'{distance}', (26, 198, 50), 20, 255
#     im = draw_txt(im, fnt, point, word, color, size, ops)
    ####  type 
    distance = 'TYPE'
    point, word, color, size, ops = (130, h-990), f'{distance}', (26, 198, 50), 20, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    ####  type 
    distance = type_tr
    point, word, color, size, ops = (35, h-975), f'{distance}', (255, 255, 255), 70, 255
    im = draw_txt(im, fnt, point, word, color, size, ops)
    
    im.show()
#    im = numpy.array(im) 
    return im    

if __name__ =="__main__":
    im = Image.open("data/res/im2_test.jpg").convert("RGBA")
    im2 = Image.open("data/res/watermark/ci.png").convert("RGBA")
   # main_draw(im, im2, distance=0, velocity=0, angle=0, carry_d = 0, t_time = 0 , spin_rpm= 0, height_tr = 0, type_tr= "none")
    main_draw(im, im2, distance=129, velocity=78, angle=20, carry_d = 121, t_time = 4.23 , spin_rpm= 3200, height_tr = 34, type_tr= "stright")


In [15]:
import math



def vel_comps(a,v):
    rad_x = math.radians(a)
    rad_y = math.radians(90-a)
    vx = v*math.cos(rad_x)
    vy = v*math.cos(rad_y)
    return vx, vy

def trav_time(y_ini):
    time_to_zero_y = y_ini/32.2
    total_time = time_to_zero_y*2
    return total_time

def trav_dist(ttime, xvel):
    dist = ttime*xvel
    return dist

def main_dist_calc(angle,vel):
    vel_x, vel_y = vel_comps(angle,vel)
    tot_time = trav_time(vel_y)
    distance = trav_dist(tot_time, vel_x)
    return tot_time, distance
if __name__ == "__main__":
    
    guide_angles = [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]
    guide_RPM = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500]

    
    angle_list = [15, 20, 25, 30,    30, 20, 20, 20, ]
    vel_list =   [87, 87, 87, 87,    70, 70, 60, 50, ]
    for angle, vel in zip(angle_list,vel_list):
        
        tot_time, distance = main_dist_calc(angle,vel)
        print(tot_time, distance)

1.3985873865788387 117.5310559006211
1.8481833831883967 151.0950129725906
2.283713588288251 180.0680245331535
2.7018633540372674 203.5697602870937
2.173913043478261 131.78647448893634
1.4870441014159512 97.81550582186469
1.2746092297851013 71.8644532568802
1.0621743581542509 49.90587031727791


ERROR: Could not find a version that satisfies the requirement odespy (from versions: none)
ERROR: No matching distribution found for odespy


In [25]:
s = "1234"
print(s[0])

1


In [25]:
def check_angle_spin(angle, vel, dist, height):
    spin = abs(angle-8)*250 + 1000

    if spin <= 2500 and vel >=100:
        dist += 55
    elif spin <= 2500 and vel <100 and vel >= 70:
        dist += 25
    elif spin <= 2500 and vel <70:
        dist += 1
        
    elif spin > 2500 and spin < 4500 and vel >=100:
        dist += 15
        height +=10
    elif spin < 2500 and spin < 4500 and vel <100 and vel >= 70:
        dist += 7
        height+=10
    elif spin < 2500 and spin < 4500 and vel <70:
        dist += 5
        height+=5
        
    elif spin >= 4500 and vel >=100:
        dist -= 35
        height+=15
    elif spin >= 4500 and vel <100 and vel >= 70:
        dist -= 12
        height+=15
    elif spin >= 4500 and vel <70:
        dist += 9
        height+=10
    return dist, height
guide_RPM = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500]
vel_list =   [60, 65, 70, 75, 80, 85, 90, 95, 100, 105]
# di_list = list_angle
# for spin, vel , dist in zip(angle_list, vel_list, di_list):
#     print(dist)
#     dist = check_angle_spin(spin, vel, dist)
#     print(dist)      
#     elif angle =< 10:
#         spin = 1500
#     elif angle =< 12:
#         spin = 2000
#     elif angle =< 14:
#         spin = 2500    
#     elif angle =< 16:
#         spin = 3000

In [40]:
guide_angles = [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]
guide_angles = [15, 20, 25, 30,    30, 20, 20, 20, ]

for angle in guide_angles:
    spin = abs(angle-8)*250 + 2000

    print(spin)


3750
5000
6250
7500
7500
5000
5000
5000


In [44]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt



def main_dist_time(vel, angle):
    spin = abs(angle-8)*250 + 2000


    # Drag coefficient, projectile radius (m), area (m2) and mass (kg).
    c = 0.47           # drag coefficent -> An average golf ball will have a drag coefficient of, 0.24 < CD < 0.7
    r = 0.02135         # radius diameter(m) = 42.7mm --> r=d/2 convert to meter = 0.02135
    A = np.pi * r**2    # area
    m = 0.04593         # mass(kg), golf_ball = 45.93 grams

    # Air density (kg.m-3), acceleration due to gravity (m.s-2).
    rho_air = 1.28
    g = 9.81

    # For convenience, define  this constant.
    k = 0.5 * c * rho_air * A

    # Initial speed and launch angle (from the horizontal).
    v0 =  vel #170
    phi0 = np.radians(angle)

    def deriv(t, u):
        x, xdot, z, zdot = u
        speed = np.hypot(xdot, zdot)
        xdotdot = -k/m * speed * xdot
        zdotdot = -k/m * speed * zdot - g
        return xdot, xdotdot, zdot, zdotdot

    # Initial conditions: x0, v0_x, z0, v0_z.
    u0 = 0, v0 * np.cos(phi0), 0., v0 * np.sin(phi0)

    # Integrate up to tf unless we hit the target sooner.
    t0, tf = 0, 50

    def hit_target(t, u):
        # We've hit the target if the z-coordinate is 0.
        return u[2]

    # Stop the integration when we hit the target.
    hit_target.terminal = True

    # We must be moving downwards (don't stop before we begin moving upwards!)
    hit_target.direction = -1

    def max_height(t, u):
        # The maximum height is obtained when the z-velocity is zero.
        return u[3]

    soln = solve_ivp(deriv, (t0, tf), u0, dense_output=True,
                     events=(hit_target, max_height))
#     print(f'velocity: {vel}, angle: {angle}')
# #     print(soln)
#     print('Time to target = {:.2f} s'.format(soln.t_events[0][0]))
#     print('Time to highest point = {:.2f} s'.format(soln.t_events[1][0]))

    # A fine grid of time points from 0 until impact time.
    t = np.linspace(0, soln.t_events[0][0], 1000)

    # Retrieve the solution for the time grid and plot the trajectory.
    sol = soln.sol(t)
    x, z = sol[0], sol[2]
    print('Range to target, xmax = {:.2f} m'.format(x[-1]))
    print('Maximum height, zmax = {:.2f} m'.format(max(z)))
    print('.......................')
    plt.plot(x, z)
    plt.xlabel('x /m')
    plt.ylabel('z /m')
    plt.show()
    top_h = max(z)
    spin_x = 1
    spin_y = 1
    dif_x = 1
    dif_x2 = 1
    # creating .obj (3D) file...
    f = open("trajectory.obj", "w")
    for i in range(x.shape[0]):
    #     f.write(f"v {x[i]} 0 {z[i]}\n")
    #     cur_m = cur_m/1.045
    #     cur_n = cur_n*1.0002
        dif_x2 /= 1.045
        dif_x *= 1.0002
        spin_y = abs(int(spin_y + dif_x+dif_x2))
#         print(spin_y)
#         print(z[i])
        f.write(f"v {x[i]} {-z[i]/10} {z[i]}\n")
    f.close()
#     x[-1], top_h = check_angle_spin(angle, vel, x[-1], top_h)
    print(x[-1], top_h)
    return x[-1], top_h



In [46]:
if __name__ == "__main__":
    vel, angle = 80, 15
    distance, height = main_dist_time(vel*2, angle)
    print(distance, height)

209.39370293011487 25.257818868794747
209.39370293011487 25.257818868794747


In [32]:
import csv

if __name__ == "__main__":
    
    
    guide_angles = [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]
    guide_RPM = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500]
    vel_list =   [60, 65, 70, 75, 80, 85, 90, 95, 100, 105]

    
    angle_list = [15, 20, 25, 30,    30, 20, 20, 20, ]
    vel_list =   [87, 87, 87, 87,    70, 70, 60, 50, ]
    vel_list =   [87, 87, 87, 87, 87, 87, 87, 87, 87, 87]
    vel_list =   [60, 65, 70, 75, 80, 85, 90, 95, 100, 105]
    vel_list2 =   [40, 45, 50, 55, 60, 65, 70, 75, 80, 85]
    
#     vel = 88
#     angle = 17.7
#     list_angle= []
    row_list = []
    row_list2 = []
    for vel in vel_list:
#         print(vel)
        list_angle= []
        list_angle2= []
        for angle in  guide_angles:
            distance, height = main_dist_time(vel, angle)
#             print(height)
            list_angle.append(int(distance))
            list_angle2.append(int(height))
#             dist = check_angle_spin(angle, vel, distance)

#         print(vel, dist)
        row_list.append(list_angle)
        row_list2.append(list_angle2)
        
header = "Velocity ", "Angle"
with open('test.csv', 'w') as fp:
    writer = csv.writer(fp, delimiter=',')
    writer.writerow(header)
    for row in row_list:
        writer.writerow([i for i in row])
        
with open('test2.csv', 'w') as fp:
    writer = csv.writer(fp, delimiter=',')
    writer.writerow(header)
    for row in row_list2:
        writer.writerow([i for i in row])

64.58456368776599 2.7713705901953953
74.24521924609347 4.116666053444137
82.49586177222841 5.653897375451594
89.58778719750788 7.359005429446411
95.70243003996355 9.211755178397894
100.97244774624713 11.19428641502307
105.50012208906492 13.290782076322856
109.36375308398772 15.487010955777144
112.62301186329968 17.769928315804354
115.32509723352265 20.1273899418247
71.72768762414694 3.1450275680504074
81.9489322217241 4.649430436149475
90.59586494572201 6.359237768601892
97.97150851879732 8.24769156288988
104.28811447999827 10.292128234044888
109.70155325427906 12.473060172706415
114.3247094404406 14.773338594654087
118.24299168454185 17.177561868491047
121.5262556426823 19.671604280860407
124.22463154274466 22.242275159330887
78.7266240414659 3.524927846480735
89.42671908613305 5.186802159540575
98.4042837289119 7.066475273088374
106.01106279230918 9.134257866624825
112.49053548629368 11.365485822794136
118.01194156210101 13.739132474797005
122.70025300085882 16.236703979473525
126.65